# Wstęp do uczenia maszynowego. Projekt nr 1.

*Maciej Borkowski, Michał Chęć
21.04.2023r.*

Zadaniem projektowym jest zrealizowanie zadania klasyfikacji binarnej na zbiorze danych numerycznych ze strony
[https://www.kaggle.com/datasets/nextbigwhat/dataset-1](https://www.kaggle.com/datasets/nextbigwhat/dataset-1)

In [4]:
# ładujemy potrzebne pakiety
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, SelectFromModel, SequentialFeatureSelector, RFE, VarianceThreshold
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

import warnings
import os
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

np.random.seed(42)

# 1. Import i podział danych

In [2]:
# wczytujemy całą ramkę danych
dataset = pd.read_csv("./data/dataset_1.csv")
df = dataset.copy()

In [3]:
# dzielimy dane, tworzymy zbiór do ewaluacji i do testów
train_set, eval_set = train_test_split(df, test_size=0.3, random_state=42)
train_df, test_df = train_test_split(train_set, test_size=0.3, random_state=42)
df = train_df.copy()

In [4]:
# sprawdzamy czy zmienna objaśniana ma mniej więcej ten sam współczynnik 1 w obu zbiorach
[dataset[dataset.target == 1].size/dataset.size,
 eval_set[eval_set.target == 1].size/eval_set.size,
 train_set[train_set.target == 1].size/train_set.size]

[0.03982, 0.04006666666666667, 0.039714285714285716]

# 2. EDA i preprocessing

W tej części zapoznajemy się ze zbiorem danych, dokonujemy eksploracyjnej analizy danych i ich preprocessingu. Należy zaznaczyć, że opisany w tej częsci preprocessing jest dokonany jedynie na pewnych wybranych przez nas parametrach, które będziemy następnie dopiero stroić w zależności od modelu. Rezultatem preprocessingu z tej części jest klasa, której będziemy używać w pipe'ach i column transformerach.

In [5]:
df.shape

(24500, 301)

In [6]:
# sprawdzamy typy zmiennych i występowanie braków
df.iloc[:, 0:100].info()
df.iloc[:, 100:200].info()
df.iloc[:, 200:300].info()
df.target.info()
df.target.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24500 entries, 10182 to 15687
Data columns (total 100 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   var_1    24500 non-null  int64  
 1   var_2    24500 non-null  int64  
 2   var_3    24500 non-null  float64
 3   var_4    24500 non-null  float64
 4   var_5    24500 non-null  float64
 5   var_6    24500 non-null  int64  
 6   var_7    24500 non-null  int64  
 7   var_8    24500 non-null  int64  
 8   var_9    24500 non-null  int64  
 9   var_10   24500 non-null  int64  
 10  var_11   24500 non-null  float64
 11  var_12   24500 non-null  float64
 12  var_13   24500 non-null  float64
 13  var_14   24500 non-null  int64  
 14  var_15   24500 non-null  int64  
 15  var_16   24500 non-null  float64
 16  var_17   24500 non-null  float64
 17  var_18   24500 non-null  float64
 18  var_19   24500 non-null  int64  
 19  var_20   24500 non-null  int64  
 20  var_21   24500 non-null  float64
 21  var_22 

0    23560
1      940
Name: target, dtype: int64

In [7]:
# odsetek 1 w zmiennej objaśnianej
df[df.target == 1].size/df.size

0.03836734693877551

Wniosek: mamy do czynienia z 300 zmiennymi numerycznymi bez braków danych, a zmienna objaśniana jest binarna. Mamy ponadto do czynienia z dość niezbalansowanym zbiorem danych.


In [8]:
df.head(10)

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300,target
10182,0,0,0.0,5.94,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
33265,0,0,0.0,0.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
8751,0,0,0.0,5.64,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
3271,0,0,0.0,2.73,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
4666,0,0,0.0,0.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
10573,0,0,0.0,6.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
22670,0,0,0.0,5.70,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
4695,0,0,0.0,2.94,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
2267,0,0,0.0,2.94,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.00,0
37603,0,0,0.0,5.70,0.0,0,0,0,0,0,...,0.0,0,0,3,0,0,0,0.0,215591.01,0


In [9]:
# rzucamy okiem na podsumowanie atrybutów numerycznych
df.describe()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300,target
count,24500.000000,24500.0,24500.000000,24500.000000,24500.000000,24500.0,24500.000000,24500.000000,24500.000000,24500.0,...,24500.00000,24500.000000,24500.0,24500.000000,24500.000000,24500.0,24500.000000,24500.000000,2.450000e+04,24500.000000
mean,0.001959,0.0,9.505698,3.168856,576.883608,0.0,0.000122,0.032816,0.000367,0.0,...,0.09703,0.009184,0.0,0.126000,0.085959,0.0,0.003143,17.845268,5.901528e+03,0.038367
std,0.101402,0.0,728.074414,2.768712,10151.771380,0.0,0.019166,0.312051,0.033196,0.0,...,1.26670,0.198049,0.0,0.604217,0.617478,0.0,0.079991,1084.015022,4.991740e+04,0.192085
min,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
50%,0.000000,0.0,0.000000,2.850000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
75%,0.000000,0.0,0.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
max,9.000000,0.0,101195.473500,35.280000,418500.000000,0.0,3.000000,3.000000,3.000000,0.0,...,74.52000,12.000000,0.0,6.000000,27.000000,0.0,3.000000,134666.681100,2.857673e+06,1.000000


Obserwujemy zmienne składające się w większości z zer z przeważnie bardzo dużą średnią i odchyleniem standardowym, w przewarzającej większości trzeci kwartyl jest zerem.

## Analiza danych całkowitoliczbowych

In [10]:
df_integer = df.loc[:, df.dtypes == "int64"].copy()
df_integer.shape

(24500, 174)

In [11]:
# sprawdzamy liczbę unikalnych wartości w kolumnach
df_integer.nunique()

var_1      4
var_2      1
var_6      1
var_7      2
var_8      2
          ..
var_295    3
var_296    7
var_297    1
var_298    4
target     2
Length: 174, dtype: int64

Sprawdzamy jakie wartości mają zmienne o typie całkowitoliczbowym

In [12]:
def show_variables(df):
    for column in df.columns:
        print(df[column].value_counts().reset_index())

show_variables(df_integer)

   index  var_1
0      0  24489
1      3      7
2      6      3
3      9      1
   index  var_2
0      0  24500
   index  var_6
0      0  24500
   index  var_7
0      0  24499
1      3      1
   index  var_8
0      0  24232
1      3    268
   index  var_9
0      0  24497
1      3      3
   index  var_10
0      0   24500
   index  var_14
0      0   24500
   index  var_15
0      3   16197
1      0    8241
2      6      61
3      9       1
   index  var_19
0      0   24257
1      1     243
   index  var_20
0      0   24496
1    450       1
2   1260       1
3    300       1
4    150       1
   index  var_23
0      0   24500
   index  var_24
0      0   24473
1      3      14
2      6       6
3      9       4
4     12       1
5     15       1
6     18       1
   index  var_25
0      0   24368
1      3     118
2      6      10
3      9       2
4     12       1
5     24       1
   index  var_28
0      0   24497
1      3       3
   index  var_31
0      3   21102
1      0    2928
2      6     44

Wnioski: istnieje bardzo wiele kolumn składających się praktycznie z jednej wartości, ponadto mogą istnieć zdublowane kolumny, kolumny o typie całkowitoliczbowym składają się zazwyczaj z niewielkiego zbioru wartości i często są binarne.

Mamy podejrzenie, że wartości w kolumnach całkowitoliczbowych reprezentują pewne kategorie. Będziemy potem kodować je za pomocą one hot encodera.

In [13]:
# poniższa funkcja podaje kolumny z df, które składają się w percent% z jednej wartości
def find_constant_columns(df, percent):

    res = []
    for column in df.columns:
        if df[column].value_counts(normalize=True).iloc[0] >= percent/100:
            res.append(column)
    return res

# poniższa funkcja usuwa znalezione powyżej kolumny
def delete_constant_columns(df, percent):
    before = df.shape[1]
    df.drop(df.loc[:, find_constant_columns(df, percent)], axis=1, inplace=True)
    after = df.shape[1]
    print(f"Usunięto {before - after} kolumn, pozostało {after}.")

Usuwamy stałe i prawie stałe kolumny

In [14]:
print(find_constant_columns(df_integer, 99.99))
delete_constant_columns(df_integer, 99.99)

['var_2', 'var_6', 'var_7', 'var_10', 'var_14', 'var_23', 'var_33', 'var_34', 'var_36', 'var_44', 'var_61', 'var_66', 'var_67', 'var_69', 'var_71', 'var_72', 'var_73', 'var_80', 'var_81', 'var_87', 'var_92', 'var_97', 'var_99', 'var_104', 'var_111', 'var_112', 'var_113', 'var_116', 'var_120', 'var_122', 'var_124', 'var_127', 'var_129', 'var_133', 'var_135', 'var_141', 'var_151', 'var_158', 'var_167', 'var_170', 'var_171', 'var_182', 'var_183', 'var_187', 'var_189', 'var_195', 'var_196', 'var_201', 'var_212', 'var_215', 'var_217', 'var_223', 'var_225', 'var_233', 'var_234', 'var_247', 'var_248', 'var_285', 'var_287', 'var_289', 'var_294', 'var_297']
Usunięto 62 kolumn, pozostało 112.


Usuwamy ewentualne zduplikowane kolumny

In [15]:
def find_duplicated_columns(df):
    return df.loc[:, df.T.duplicated()].columns.tolist()

def delete_duplicated_columns(df):
    before = df.shape[1]
    df.drop(df.loc[:, df.T.duplicated()], axis=1, inplace=True)
    after = df.shape[1]
    print(f"Usunięto {before - after} kolumn, pozostało {after}.")

In [16]:
print(find_duplicated_columns(df_integer))
delete_duplicated_columns(df_integer)

['var_106', 'var_148', 'var_197', 'var_199', 'var_216', 'var_232', 'var_239', 'var_250', 'var_263', 'var_269', 'var_296']
Usunięto 11 kolumn, pozostało 101.


In [17]:
# poniższa funkcja podaje kolumny z [data], które są w [percent]% zależne od innej kolumny (spośród pary skorelowanych kolumn podawana jest jedna)
def find_corelated_columns(data, percent,mark):

    ### mark = 1 dla float
    ### cokolwiek dla pseudodyskretnych
    
    if(mark == 1):
        corr = data.corr(method='pearson')
    else:
        corr = data.corr(method='spearman')
    corr = corr[corr > percent/100]

    dependent_columns = corr.apply(lambda row: row[row > 0].index, axis=1)
    res = []
    for j in range (len(dependent_columns)):
        for k in dependent_columns[j]:
            if k is not dependent_columns.index[j]:
                if k not in dependent_columns.index[0:j]:
                    res.append(k)

    return np.unique(np.array(res))

def delete_corelated_columns(df, percent,mark):
    before = df.shape[1]
    df.drop(find_corelated_columns(df, percent,mark), axis=1, inplace=True)
    after = df.shape[1]
    print(f"Usunięto {before - after} kolumn, pozostało {after}.")

Usuwamy kolumny wysoce skorelowane z innymi

In [18]:
print(find_corelated_columns(df_integer, 99.99,-1))
delete_corelated_columns(df_integer, 99.99,-1)

['var_114' 'var_160' 'var_163' 'var_172' 'var_177' 'var_198' 'var_211'
 'var_229' 'var_235' 'var_238' 'var_249' 'var_251' 'var_258' 'var_259'
 'var_260' 'var_267' 'var_270' 'var_273' 'var_275' 'var_281' 'var_282'
 'var_291' 'var_295' 'var_53' 'var_54' 'var_88']
Usunięto 26 kolumn, pozostało 75.


Sprawdzamy jak teraz prezentują się wartości w zmiennych całkowitoliczbowych

In [19]:
show_variables(df_integer)

   index  var_1
0      0  24489
1      3      7
2      6      3
3      9      1
   index  var_8
0      0  24232
1      3    268
   index  var_9
0      0  24497
1      3      3
   index  var_15
0      3   16197
1      0    8241
2      6      61
3      9       1
   index  var_19
0      0   24257
1      1     243
   index  var_20
0      0   24496
1    450       1
2   1260       1
3    300       1
4    150       1
   index  var_24
0      0   24473
1      3      14
2      6       6
3      9       4
4     12       1
5     15       1
6     18       1
   index  var_25
0      0   24368
1      3     118
2      6      10
3      9       2
4     12       1
5     24       1
   index  var_28
0      0   24497
1      3       3
   index  var_31
0      3   21102
1      0    2928
2      6     445
3      9      22
4     12       2
5     33       1
   index  var_37
0      0   23935
1      3     485
2      6      65
3      9      10
4     12       3
5     27       1
6     21       1
   index  var_38
0      0

Jak wartości ze zmiennych całkowitoliczbowych mają się do zmiennej target?

In [20]:
def show_dependencies(df):
    for column in df.columns:
        print(df.groupby(column)['target'].agg(['sum','count']).sort_values('sum',ascending = False))

In [21]:
show_dependencies(df_integer)

       sum  count
var_1            
0      940  24489
3        0      7
6        0      3
9        0      1
       sum  count
var_8            
0      936  24232
3        4    268
       sum  count
var_9            
0      940  24497
3        0      3
        sum  count
var_15            
0       617   8241
3       321  16197
6         2     61
9         0      1
        sum  count
var_19            
0       939  24257
1         1    243
        sum  count
var_20            
0       940  24496
150       0      1
300       0      1
450       0      1
1260      0      1
        sum  count
var_24            
0       940  24473
3         0     14
6         0      6
9         0      4
12        0      1
15        0      1
18        0      1
        sum  count
var_25            
0       936  24368
3         4    118
6         0     10
9         0      2
12        0      1
24        0      1
        sum  count
var_28            
0       940  24497
3         0      3
        sum  count
var_31 

Obserwujemy że występują kolumny, w których cała informacja o jedynkach jest niesiona przez wartość 0. Znajdźmy i zlikwidujmy je.

In [22]:
def find_not_informational_columns(df, x):
    amount_of_ones = df[df.target == 1].shape[0]
    res = []
    for column in df.columns:
        tmp = df.groupby(column)['target'].agg(['sum','count']).sort_values('sum',ascending = False).reset_index()
        if any(tmp[column] == 0) and (tmp.loc[tmp[column] == 0, 'sum'] >= amount_of_ones - x).bool():
            res.append(column)
    return res

def delete_not_informational_columns(df, x):
    before = df.shape[1]
    df.drop(find_not_informational_columns(df, x), axis=1, inplace=True)
    after = df.shape[1]
    print(f"Usunięto {before - after} kolumn, pozostało {after}.")

In [23]:
print(find_not_informational_columns(df_integer, 1))
delete_not_informational_columns(df_integer, 1)

['var_1', 'var_9', 'var_19', 'var_20', 'var_24', 'var_28', 'var_43', 'var_45', 'var_51', 'var_56', 'var_59', 'var_60', 'var_68', 'var_137', 'var_146', 'var_149', 'var_159', 'var_181', 'var_194', 'var_218', 'var_219', 'var_221', 'var_236', 'var_245', 'var_256', 'var_274', 'var_278', 'var_298']
Usunięto 28 kolumn, pozostało 47.


In [24]:
show_dependencies(df_integer)

       sum  count
var_8            
0      936  24232
3        4    268
        sum  count
var_15            
0       617   8241
3       321  16197
6         2     61
9         0      1
        sum  count
var_25            
0       936  24368
3         4    118
6         0     10
9         0      2
12        0      1
24        0      1
        sum  count
var_31            
3       755  21102
0       164   2928
6        20    445
9         1     22
12        0      2
33        0      1
        sum  count
var_37            
0       904  23935
3        33    485
6         3     65
9         0     10
12        0      3
21        0      1
27        0      1
        sum  count
var_38            
0       935  23577
3         5    923
        sum  count
var_49            
99      612   9685
1       145   4693
3       106   7183
2        77   2808
0         0    131
        sum  count
var_52            
0       931  23374
1         9   1126
        sum  count
var_58            
0       928  228

Powyższymi zabiegami zredukowaliśmy liczbę kolumn całkowitoliczbowych z 174 do 55, oczywiście liczba ta wzrośnie, kiedy będziemy chcieli je zakodować za pomocą one hot encodera.

## Analiza danych zmiennoprzecinkowych

In [25]:
df_float = df.loc[:, df.dtypes == "float64"].copy()
df_float = df_float.join(df.target)
df_float.shape

(24500, 128)

In [26]:
df_float.describe()

,var_3,var_4,var_5,var_11,var_12,var_13,var_16,var_17,var_18,var_21,...,var_279,var_280,var_283,var_286,var_288,var_290,var_292,var_299,var_300,target
count,24500.000000,24500.000000,24500.000000,24500.0,24500.0,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,...,24500.000000,24500.0,24500.0,24500.000000,24500.000000,24500.000000,24500.00000,24500.000000,2.450000e+04,24500.000000
mean,9.505698,3.168856,576.883608,0.0,0.0,5.049415,56.243582,1.147002,104.985083,1035.879430,...,2.799549,0.0,0.0,82.450338,0.110804,10.246986,0.09703,17.845268,5.901528e+03,0.038367
std,728.074414,2.768712,10151.771380,0.0,0.0,231.507148,2878.129876,3.120918,1902.742951,9442.807758,...,10.460820,0.0,0.0,3153.057895,1.083260,533.527182,1.26670,1084.015022,4.991740e+04,0.192085
min,0.000000,0.000000,0.000000,0.0,0.0,-0.900000,0.000000,0.000000,-685.475100,-464.544000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000
50%,0.000000,2.850000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.910000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000
75%,0.000000,3.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,86.400000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000
max,101195.473500,35.280000,418500.000000,0.0,0.0,33120.000000,377990.085000,49.980000,158256.974400,552414.000000,...,253.800000,0.0,0.0,246477.390600,41.850000,45503.530800,74.52000,134666.681100,2.857673e+06,1.000000


Sprawdzamy rozkłady zmiennych o typie zmiennoprzecinkowym

In [27]:
# for column in df_float.columns:
#     df_float[column].hist(bins=10, figsize=(5, 2))
#     plt.show()

Wniosek: kolumny o wartościach zmiennoprzecinkowych składają się głównie z samych 0 i mają dość nietypowe rozkłady

Korzystając ze zdefiniowanych funkcji usuwamy stałe i prawie stałe kolumny, zduplikowane kolumny, zależne od innych i nie niosące informacji o 1

In [28]:
delete_duplicated_columns(df_float)
delete_constant_columns(df_float, 99.99)
delete_corelated_columns(df_float, 99.99,1)
delete_not_informational_columns(df_float, 10)

Usunięto 10 kolumn, pozostało 118.
Usunięto 4 kolumn, pozostało 114.
Usunięto 1 kolumn, pozostało 113.
Usunięto 57 kolumn, pozostało 56.


Łączymy ramki danych float i int

In [29]:
df_float.drop('target', axis=1, inplace=True)
df = df_integer.join(df_float)
df.shape

(24500, 102)

In [30]:
df.describe()

,var_8,var_15,var_25,var_31,var_37,var_38,var_49,var_52,var_58,var_62,...,var_255,var_261,var_266,var_271,var_272,var_276,var_277,var_279,var_288,var_292
count,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,...,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.000000,24500.00000
mean,0.032816,1.998612,0.019102,2.703306,0.082408,0.113020,40.435633,0.045959,0.208286,2.726327,...,63.376736,1.629343,34.968528,0.179449,1.169796,760.923713,2.108832,2.799549,0.110804,0.09703
std,0.312051,1.431255,0.306071,1.107591,0.604439,0.571228,47.357524,0.209401,0.785344,1.136362,...,382.677763,7.488873,371.785970,0.966962,4.864488,12060.271412,8.616134,10.460820,1.083260,1.26670
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,2.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,3.000000,0.000000,3.000000,0.000000,0.000000,3.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,0.000000,3.000000,0.000000,3.000000,0.000000,0.000000,99.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,3.000000,9.000000,24.000000,33.000000,27.000000,3.000000,99.000000,1.000000,15.000000,33.000000,...,12672.000000,226.590000,26289.421800,43.200000,107.160000,787920.000000,247.950000,253.800000,41.850000,74.52000


### Variance Threshold

Po takiej selekcji, nasze cechy możemy dodatkowo ograniczyć ze względu na wariancję. 
Użyjemy do tego poniższej funkcji:

In [31]:
def rem_by_Var_threshold(df, threshold):
    X = df.drop('target', axis=1)
    
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(X)
    selected_features = X.columns[selector.get_support()]
    X_selected = X[selected_features]
    return X_selected.join(df.target)

Patrząc na wariancje za pomocą powyższej funkcji możemy usunąć cechy, które mają ją za małą - niosą ze sobą bardzo mało informacji. Nie wykorzystujemy jednak tej funkcji w naszym zbiorze danych - zbadaliśmy że nie daje satysfakcjonujących wyników.

## Podsumowanie preprocessingu

### Co z kodowaniem zmiennych całkowitoliczbowych i skalowanie zmiennych?

Oczywiście nie zapomnieliśmy o tych krokach, będziemy je natomiast uskuteczniać w zależności od dobranego modelu i umieścimy je w pipe'ach.

### Co z outlierami?

Traktując zmienne dyskretne jako kategoryczne i przyglądając się dziwnym i często dośc losowym rozkładom zmiennych ciągłych nie jesteśmy w stanie stwierdzić co jest tak naprawdę outlierem. Nie pomaga w tym również brak wiedzy eksperckiej z zakresu badanego problemu i brak wiedzy na temat znaczenia cech zbioru danych.

### A inżynieria i dobór cech?

Oprócz zawartej już w preprocessingu analizy korelacji, inżynierię i dobór cech przeprowadzimy dla każdego modelu oddzielnie.

### Co udało się zrealizować

Zapoznaliśmy się ze zbiorem danych. Do tej pory znajdowaliśmy kolumny składające się z jednej lub prawie jednej wartości, kolumny zduplikowane, skorelowane i nie niosące informacji o 1. Oczywiście niektóre z naszych podejść może okazać się błędne, więc opakowaliśmy procedury procesujące w funkcje, tak aby dobierać ich parametry w zależności od modelu.

Cały opisany dotychczas preprocessing zawieramy w transformatorze ColumnRemover - dzięki utworzeniu osobnej klasy procesującej będziemy mogli używać go w pipe'ach i column transformerach i dobierać parametry za pomocą grid search.

In [ ]:
class ColumnRemover(BaseEstimator, TransformerMixin):

    def __init__(self, threshold_constant, threshold_corr, n_info_vals):
        self.threshold_constant = threshold_constant
        self.threshold_corr = threshold_corr
        self.n_info_vals = n_info_vals
        self.columns_to_remove = []
        self.columns_to_keep = []

    def fit(self, X, y=None):
        # usuwanie zduplikowanych kolumn
        self.columns_to_remove.extend(X.loc[:, X.T.duplicated()].columns.tolist())

        # usuwanie stałych i prawie stałych kolumn
        for column in X.columns:
            if X[column].value_counts(normalize=True).iloc[0] >= self.threshold_constant:
                self.columns_to_remove.append(column)

        # usuwanie skorelowanych kolumn, jeśli pierwsza kolumna jest ciągła stosujemy korelację pearsona,
        # jeśli dyskretna to korelację spearmana; pierwsza kolumna będzie typu takiego samego jak cały X ponieważ
        # klasę tą stosujemy w column tranformerze ograniczając podzbiór kolumn do okreslonego typu
        if X.dtypes[0] == 'float64':
            corr = X.corr(method='pearson')
        else:
            corr = X.corr(method='spearman')
        corr = corr[corr > self.threshold_corr]
        dependent_columns = corr.apply(lambda row: row[row > 0].index, axis=1)
        for j in range (len(dependent_columns)):
            for k in dependent_columns[j]:
                if k is not dependent_columns.index[j]:
                    if k not in dependent_columns.index[0:j]:
                        self.columns_to_remove.append(k)

        # usuwanie kolumn nie niosących informacji
        amount_of_ones = y[y == 1].shape[0]
        X = X.join(y)
        for column in X.columns:
            tmp = X.groupby(column)['target'].agg(['sum','count']).sort_values('sum',ascending = False).reset_index()
            if any(tmp[column] == 0) and (tmp.loc[tmp[column] == 0, 'sum'] > amount_of_ones - self.n_info_vals).bool():
                self.columns_to_remove.append(column)
        X.drop('target', axis=1, inplace=True)

        self.columns_to_keep = [col for col in X.columns if col not in self.columns_to_remove]

        return self

    def transform(self, X):
        return X[self.columns_to_keep]

Utworzyliśmy ponadto funkcję liczącą wszystko za nas i wyświetlającą macierz pomyłek oraz interesujące nas metryki

In [33]:
def show_scores(clf, X, y):
    y_pred = clf.predict(X)
    y_pred_prob = clf.predict_proba(X)
    print(tabulate(confusion_matrix(y, y_pred), headers=['Predicted 0', 'Predicted 1'], tablefmt='orgtbl'))
    print()
    print(f'accuracy:              {round(accuracy_score(y, y_pred), 4)}')
    print(f'precision:             {round(precision_score(y, y_pred), 4)}')
    print(f'recall:                {round(recall_score(y, y_pred), 4)}')
    print(f'f1:                    {round(f1_score(y, y_pred), 4)}')
    print(f'roc_auc_discrete:      {round(roc_auc_score(y, y_pred), 4)}')
    print(f'roc_auc_continuous:    {round(roc_auc_score(y, y_pred_prob[:, 1]), 4)}')

Przejdźmy zatem do modeli

# 3. Regresja logistyczna

## 3.1 Preprocessing

In [34]:
# zarówno dla zmiennych dyskretnych i ciągłych stosujemy nasz transformator ColumnRemover z różnymi parametrami - w kolejnych krokach będziemy szukać najlepszej ich kombinacji

# dokonujemy kodowania one hot encoding zmiennych dyskretnych - traktujemy je jako kategoryczne
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

# dokojumeny standaryzacji zmiennych ciągłych - sprawdziliśmy, że jest lepsza od normalizacji min-max dla tego modelu
float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10)),
    ('standard_scaler', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

## 3.2 Trening pierwszego modelu

In [35]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
X_test = test_df.drop('target', axis=1)
y_test = test_df.target

In [36]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', LogisticRegression(random_state=42))])

clf.fit(X_train, y_train)

# wyniki dla danych treningowych
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23550 |            10 |
|           934 |             6 |

accuracy:              0.9615
precision:             0.375
recall:                0.0064
f1:                    0.0126
roc_auc_discrete:      0.503
roc_auc_continuous:    0.8057


In [37]:
# wyniki dla danych testowych
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         10033 |            17 |
|           449 |             1 |

accuracy:              0.9556
precision:             0.0556
recall:                0.0022
f1:                    0.0043
roc_auc_discrete:      0.5003
roc_auc_continuous:    0.7813


Model się nie uczy. Wiemy, że klasy są niezbalansowane (jest ok. 4% jedynek). Zastosujemy parametr class_weight = 'balanced')

In [38]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16261 |          7299 |
|           207 |           733 |

accuracy:              0.6936
precision:             0.0913
recall:                0.7798
f1:                    0.1634
roc_auc_discrete:      0.735
roc_auc_continuous:    0.8125


In [39]:
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6841 |          3209 |
|           120 |           330 |

accuracy:              0.683
precision:             0.0932
recall:                0.7333
f1:                    0.1655
roc_auc_discrete:      0.707
roc_auc_continuous:    0.7784


Daje nam to minimalne podstawy do dalszego doboru parametrów.

## 3.3 Strojenie hiperparametrów i dobór cech

### 3.3.1 Dobór parametrów dla ColumnRemover'a


In [40]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
X_test = test_df.drop('target', axis=1)
y_test = test_df.target

In [41]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

parameters = dict(preprocessing__int_pipe__int__threshold_constant = np.arange(0.9995, 1, 0.0001),
                  preprocessing__int_pipe__int__threshold_corr = np.arange(0.96, 1, 0.01),
                  preprocessing__int_pipe__int__n_info_vals = np.arange(0, 5, 1),
                  preprocessing__float_pipe__float__threshold_constant = np.arange(0.9995, 1, 0.0001),
                  preprocessing__float_pipe__float__threshold_corr = np.arange(0.96, 1, 0.01),
                  preprocessing__float_pipe__float__n_info_vals = np.arange(0, 16, 3))

col_remove_search = RandomizedSearchCV(clf, scoring='roc_auc', param_distributions=parameters, cv=3, n_iter=1000, n_jobs=-1, random_state=42).fit(X_train, y_train)

Sprawdzamy rezultaty treningu

In [42]:
print(round(col_remove_search.best_score_, 4), col_remove_search.best_params_)

0.7845 {'preprocessing__int_pipe__int__threshold_corr': 1.0, 'preprocessing__int_pipe__int__threshold_constant': 0.9998, 'preprocessing__int_pipe__int__n_info_vals': 0, 'preprocessing__float_pipe__float__threshold_corr': 0.97, 'preprocessing__float_pipe__float__threshold_constant': 0.9996, 'preprocessing__float_pipe__float__n_info_vals': 0}


In [43]:
show_scores(col_remove_search.best_estimator_, X_train, y_train)
show_scores(col_remove_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16407 |          7153 |
|           205 |           735 |

accuracy:              0.6997
precision:             0.0932
recall:                0.7819
f1:                    0.1665
roc_auc_discrete:      0.7392
roc_auc_continuous:    0.8155
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6898 |          3152 |
|           121 |           329 |

accuracy:              0.6883
precision:             0.0945
recall:                0.7311
f1:                    0.1674
roc_auc_discrete:      0.7087
roc_auc_continuous:    0.776


Otrzymujemy porównywalne wyniki.

### 3.3.2 Regularyzacja modelu

Zajmiemy się regularyzacją modelu regresji liniowej - będziemy sprawdzać odwrotność współczynnika regularyzacji (im większy tym mniejsza regulacja), oraz rodzaj kary (l1 - regresja LASSO, l2 - regresja grzbietowa). Zastosujemy podadto parametry ColumnRemovera ustalone w poprzednich podpunktach.


In [44]:
# ustawiamy parametry ColumnRemovera na znalezione w 3.3.1
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9998, 1, 0)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9996, 0.97, 0)),
    ('standard_scaler', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

X_train = col_transformer.fit_transform(train_df.drop('target', axis=1), train_df.target)
y_train = train_df.target
X_test = col_transformer.transform(test_df.drop('target', axis=1))
y_test = test_df.target

In [45]:
clf = LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear')

parameters = dict(C=np.logspace(-6, 2, 20), penalty=['l1', 'l2'])
reg_search = GridSearchCV(clf, scoring='roc_auc', cv=3, return_train_score=True, param_grid=parameters, n_jobs=-1).fit(X_train, y_train)

Sprawdzamy najlepsze parametry i wyniki dla najlepszego modelu

In [46]:
print(round(reg_search.best_score_, 4), reg_search.best_params_)

0.7879 {'C': 0.11288378916846883, 'penalty': 'l1'}


In [47]:
show_scores(reg_search.best_estimator_, X_train, y_train)
show_scores(reg_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16357 |          7203 |
|           213 |           727 |

accuracy:              0.6973
precision:             0.0917
recall:                0.7734
f1:                    0.1639
roc_auc_discrete:      0.7338
roc_auc_continuous:    0.8121
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6894 |          3156 |
|           121 |           329 |

accuracy:              0.6879
precision:             0.0944
recall:                0.7311
f1:                    0.1672
roc_auc_discrete:      0.7085
roc_auc_continuous:    0.782


In [48]:
cvres = reg_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(round(mean_score, 4), "   ", params)

0.5     {'C': 1e-06, 'penalty': 'l1'}
0.7605     {'C': 1e-06, 'penalty': 'l2'}
0.5     {'C': 2.6366508987303555e-06, 'penalty': 'l1'}
0.761     {'C': 2.6366508987303555e-06, 'penalty': 'l2'}
0.5     {'C': 6.951927961775606e-06, 'penalty': 'l1'}
0.7628     {'C': 6.951927961775606e-06, 'penalty': 'l2'}
0.5     {'C': 1.8329807108324375e-05, 'penalty': 'l1'}
0.766     {'C': 1.8329807108324375e-05, 'penalty': 'l2'}
0.5     {'C': 4.8329302385717524e-05, 'penalty': 'l1'}
0.7709     {'C': 4.8329302385717524e-05, 'penalty': 'l2'}
0.5     {'C': 0.00012742749857031334, 'penalty': 'l1'}
0.7755     {'C': 0.00012742749857031334, 'penalty': 'l2'}
0.5     {'C': 0.0003359818286283781, 'penalty': 'l1'}
0.7801     {'C': 0.0003359818286283781, 'penalty': 'l2'}
0.7333     {'C': 0.0008858667904100823, 'penalty': 'l1'}
0.7832     {'C': 0.0008858667904100823, 'penalty': 'l2'}
0.7714     {'C': 0.002335721469090121, 'penalty': 'l1'}
0.7851     {'C': 0.002335721469090121, 'penalty': 'l2'}
0.7792     {'C': 0.0061

Skorzystamy z wyznaczonych najlepszych parametrów przy selekcji cech metodą lasso.

### 3.3.3 Dobór zmiennych nie wymagajacy modelu

Korelację uwzględniliśmy w transformatorze ColumnRemover. Technika którą wykorzystamy poniżej to Univariate feature selection - SelectKBest.


### SelectKBest

In [49]:
# szukamy najlepszego parametru k w SelectKBest
clf = Pipeline([
    ('select', SelectKBest()),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

parameters = dict(select__k=np.arange(1, 200, 1))
k_best_search = GridSearchCV(clf, scoring='roc_auc', cv=3, return_train_score=True, param_grid=parameters, n_jobs=-1).fit(X_train, y_train)

In [50]:
# wyniki dla najlepszego znalezionego parametru
print(round(k_best_search.best_score_, 4), k_best_search.best_params_)

0.7928 {'select__k': 154}


In [51]:
show_scores(k_best_search.best_estimator_, X_train, y_train)
show_scores(k_best_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16290 |          7270 |
|           208 |           732 |

accuracy:              0.6948
precision:             0.0915
recall:                0.7787
f1:                    0.1637
roc_auc_discrete:      0.7351
roc_auc_continuous:    0.8102
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6884 |          3166 |
|           121 |           329 |

accuracy:              0.687
precision:             0.0941
recall:                0.7311
f1:                    0.1668
roc_auc_discrete:      0.708
roc_auc_continuous:    0.7796


Wydaje się dość duże k, zobaczmy, jak prezentują się wszystkie wyniki

In [52]:
cvres = k_best_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(round(mean_score, 4), "   ", params)

0.6717     {'select__k': 1}
0.6741     {'select__k': 2}
0.6741     {'select__k': 3}
0.675     {'select__k': 4}
0.6772     {'select__k': 5}
0.6819     {'select__k': 6}
0.6822     {'select__k': 7}
0.6914     {'select__k': 8}
0.7001     {'select__k': 9}
0.7115     {'select__k': 10}
0.7108     {'select__k': 11}
0.7657     {'select__k': 12}
0.7659     {'select__k': 13}
0.7651     {'select__k': 14}
0.7779     {'select__k': 15}
0.7781     {'select__k': 16}
0.778     {'select__k': 17}
0.7785     {'select__k': 18}
0.7786     {'select__k': 19}
0.7785     {'select__k': 20}
0.78     {'select__k': 21}
0.7802     {'select__k': 22}
0.7804     {'select__k': 23}
0.7805     {'select__k': 24}
0.7806     {'select__k': 25}
0.7806     {'select__k': 26}
0.7811     {'select__k': 27}
0.7815     {'select__k': 28}
0.7816     {'select__k': 29}
0.7817     {'select__k': 30}
0.7819     {'select__k': 31}
0.7843     {'select__k': 32}
0.7844     {'select__k': 33}
0.7843     {'select__k': 34}
0.7843     {'select__k': 35

Wnioski: roc_auc maleje nieznacznie i wybierając dużo mniej zmiennych nie stracimy zbytnio na wyniku, a zmniejszymy drastycznie liczbę cech.

In [53]:
clf = Pipeline([
    ('select', SelectKBest(k=33)),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         15821 |          7739 |
|           231 |           709 |

accuracy:              0.6747
precision:             0.0839
recall:                0.7543
f1:                    0.151
roc_auc_discrete:      0.7129
roc_auc_continuous:    0.7852
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6671 |          3379 |
|           125 |           325 |

accuracy:              0.6663
precision:             0.0877
recall:                0.7222
f1:                    0.1565
roc_auc_discrete:      0.693
roc_auc_continuous:    0.7766


Obserwujemy nieznaczne pogorszenie wyników (szczególnie nieznaczne jest na zbiorze testowym), jednak zredukowaliśmy liczbę cech do zaledwie 33.

### 3.3.4 Dobór zmiennych na podstawie modelu

Dotychczasowe starania miały na celu osiągnięcie jak najlepszych rezultatów dla modelu. Teraz przeprowadzimy dobór cech na podstawie modelu ze znalezionymi najlepszymi parametrami. Zastosujemy metody L1-based feature selection, Sequential Feature Selection i RFE

### SelectFromModel - L1-based feature selection

In [55]:
clf = LogisticRegression(**reg_search.best_params_, random_state=42, solver='liblinear', class_weight='balanced')
clf.fit(X_train, y_train)

show_scores(clf, X_train, y_train)
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16357 |          7203 |
|           213 |           727 |

accuracy:              0.6973
precision:             0.0917
recall:                0.7734
f1:                    0.1639
roc_auc_discrete:      0.7338
roc_auc_continuous:    0.8121
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6894 |          3156 |
|           121 |           329 |

accuracy:              0.6879
precision:             0.0944
recall:                0.7311
f1:                    0.1672
roc_auc_discrete:      0.7085
roc_auc_continuous:    0.782


In [56]:
X_train.shape[1]

406

In [57]:
sfl = SelectFromModel(clf, prefit=True)
X_train_t = sfl.transform(X_train)
X_test_t = sfl.transform(X_test)
X_train_t.shape[1]

114

In [58]:
clf.fit(X_train_t, y_train)

show_scores(clf, X_train_t, y_train)
show_scores(clf, X_test_t, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16357 |          7203 |
|           213 |           727 |

accuracy:              0.6973
precision:             0.0917
recall:                0.7734
f1:                    0.1639
roc_auc_discrete:      0.7338
roc_auc_continuous:    0.8121
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          6894 |          3156 |
|           121 |           329 |

accuracy:              0.6879
precision:             0.0944
recall:                0.7311
f1:                    0.1672
roc_auc_discrete:      0.7085
roc_auc_continuous:    0.7823


Nie obserwujemy żadnego spadku w wynikach - zreudkowaliśmy natomiast liczbę cech do 114.

### Sequential Feature Selection

In [60]:
sfs = SequentialFeatureSelector(
    LogisticRegression(random_state=42, class_weight='balanced'),
    direction='forward',
    scoring='roc_auc',
    n_features_to_select=40,
    cv=3,
    n_jobs=-1)

pipe = Pipeline([
    ('selector', sfs),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))]).fit(X_train, y_train)

show_scores(pipe, X_train, y_train)

In [61]:
show_scores(pipe, X_test, y_test)

Po kilku próbach otrzymaliśmy niepogorszone wyniki dla zalwdwie 40 zmiennych na podstawie selekcji SequentialFeaturSelector. Nie szukamy w tym przypadku parametru n_features_to_select za pomocą grid search, ponieważ zajęło by nam to z pół roku, a zajęliśmy się empiryczym sprawdzeniem najlepszej wartości.


### Recursive Feature Elimination

Dla najlepszego modelu regresji logistycznej sprawdźmy które cechy niosą za sobą najwięcej informacji

In [62]:
k_val = [k for k in range (2,100,2)]
score = []
for k in k_val:
    print(k)
    log_reg = LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear')
    rfe = RFE(log_reg,n_features_to_select = k)
    rfe.fit(X_train, y_train)
    X_train_rfe = rfe.transform(X_train)
    X_test_rfe = rfe.transform(X_test)
    log_reg.fit(X_train_rfe, y_train)

    y_pred_prob = log_reg.predict_proba(pd.DataFrame(X_test_rfe))

    score_roc = roc_auc_score(y_test, y_pred_prob[:, 1])
    score.append(score_roc)
    print(score_roc, k)

In [64]:
plt.stem(score)
plt.xticks(k_val, score)
plt.xlim([-1, 50])
plt.xlabel("K Values")
plt.ylabel("roc_auc Score")

![fre_logreg.png](./Img/rfe_logreg.png)

Z powyższego wykresu możemy wyczytać, że najlepiej będzie użyć rfe dla 72 cech.
W związku z tym spróbujmy użyć PCA na wybranych kolumnach.

(uwaga: ten i pozostałe wykresy zawarte w tym notatniku zostały przeklejone z innych naszych notatników ze względu na długi czas liczenia kodu rfe)

### Principal Component Analysis

In [65]:
rfe = RFE(LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear'),n_features_to_select = 72).fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

k_values = [i for i in range (2,72)]
scores = []

for k in k_values:

    log_reg = LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear')
    pca = PCA(n_components=k)

    X_red_train = pca.fit_transform(X_train_rfe)
    X_red_test = pca.fit_transform(X_test_rfe)

    log_reg.fit(X_red_train, y_train)
    y_pred_prob = log_reg.predict_proba(pd.DataFrame(X_red_test))

    score_roc = roc_auc_score(y_test, y_pred_prob[:, 1])
    scores.append(score_roc)


In [66]:
plt.stem(scores)
plt.xticks(k_values, scores)
plt.xlim([-1, 72])
plt.xlabel("K Values")
plt.ylabel("roc_auc Score")
plt.title("PCA for linear regression")

![linear_reg.png](./Img/linear_reg.png)

Widzimy więc, że dla tego modelu najlepiej będzie wybrać wartość n_components = 3

### 3.4 Podsumowanie

Model regresji logistycznej działa przyzwoicie jak na warunki otrzymanego zbioru danych. Istotne wydaje się użycie parametry class_weight = 'balanced'. Przeprowadziliśmy przeszukanie siatki parametrów autorskiego ColumnRemover'a, a następnie przeszukiwanie siatki parametru C i kar w celu dobrania najlepszych parametrów regularyzacji.

Następnie przeszliśmy do selekcji cech różnymi metodami. W wielu przypadkach zaobserwowaliśmy znaczny spadek liczby kolumn przy jednoczesnym braku znaczącego pogorszenia rezultatów (nie zaobserwowaliśmy znaczącego wzrostu rezultatów przy jakiejkolwiek redukcji cech). Metody jakie wybraliśmy i ostateczna liczba zmiennych prezentują się następujaco:

- SelectKBest - 33
- L1-based feature selection - 114
- Sequential feature selection - 40
- Recursive feature elimination - 72
- RFE + PCA - 3

# 4. Drzewa decyzyjne

## 4.1 Preprocessing

In [67]:
# zarówno dla zmiennych dyskretnych i ciągłych stosujemy nasz transformator ColumnRemover z różnymi parametrami - w kolejnych krokach będziemy szukać najlepszej ich kombinacji

# dokonujemy kodowania one hot encoding zmiennych dyskretnych - traktujemy je jako kategoryczne
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

# jesteśmy przy drzewach decyzyjnych, więc nie musimy skalować cech
float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

## 4.2 Trening pierwszego modelu

In [68]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
X_test = test_df.drop('target', axis=1)
y_test = test_df.target

In [69]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', DecisionTreeClassifier(random_state=42))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0


In [70]:
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9654 |           396 |
|           384 |            66 |

accuracy:              0.9257
precision:             0.1429
recall:                0.1467
f1:                    0.1447
roc_auc_discrete:      0.5536
roc_auc_continuous:    0.5536


Model jest zdecydowanie przeuczony. Spróbujmy z parametrem class_weight='balanced'.

In [71]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', DecisionTreeClassifier(random_state=42, class_weight='balanced'))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23557 |             3 |
|             0 |           940 |

accuracy:              0.9999
precision:             0.9968
recall:                1.0
f1:                    0.9984
roc_auc_discrete:      0.9999
roc_auc_continuous:    1.0


In [72]:
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9705 |           345 |
|           399 |            51 |

accuracy:              0.9291
precision:             0.1288
recall:                0.1133
f1:                    0.1206
roc_auc_discrete:      0.5395
roc_auc_continuous:    0.5395


Za dużo to nie dało, musimy skupić się na regularyzacji modelu drzewa decyzyjnego.

## 4.3 Strojenie hiperparametrów i wybór zmiennych

### 4.3.1 Dobór parametrów dla ColumnRemover'a


In [73]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', DecisionTreeClassifier(random_state=42))])

parameters = dict(preprocessing__int_pipe__int__threshold_constant = np.arange(0.9995, 1, 0.0001),
                  preprocessing__int_pipe__int__threshold_corr = np.arange(0.96, 1, 0.01),
                  preprocessing__int_pipe__int__n_info_vals = np.arange(0, 5, 1),
                  preprocessing__float_pipe__float__threshold_constant = np.arange(0.9995, 1, 0.0001),
                  preprocessing__float_pipe__float__threshold_corr = np.arange(0.96, 1, 0.01),
                  preprocessing__float_pipe__float__n_info_vals = np.arange(0, 16, 3))

# siatka parametrów ma liczność 27000, przeszukujemy więc ok. 37% wszystkich możliwości
col_remove_search = RandomizedSearchCV(clf, scoring='roc_auc', param_distributions=parameters, cv=3, n_iter=10000, n_jobs=-1, random_state=42).fit(X_train, y_train)

In [74]:
print(round(col_remove_search.best_score_, 4), col_remove_search.best_params_)

0.5594 {'preprocessing__int_pipe__int__threshold_corr': 0.99, 'preprocessing__int_pipe__int__threshold_constant': 0.9998, 'preprocessing__int_pipe__int__n_info_vals': 2, 'preprocessing__float_pipe__float__threshold_corr': 0.96, 'preprocessing__float_pipe__float__threshold_constant': 0.9998, 'preprocessing__float_pipe__float__n_info_vals': 15}


In [75]:
show_scores(col_remove_search.best_estimator_, X_train, y_train)
show_scores(col_remove_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9653 |           397 |
|           379 |            71 |

accuracy:              0.9261
precision:             0.1517
recall:                0.1578
f1:                    0.1547
roc_auc_discrete:      0.5591
roc_auc_continuous:    0.5591


### 4.3.2 Regularyzacja modelu

Zajmiemy się regularyzacją modelu drzewa decyzyjnego - będziemy sprawdzać parametry max_depth, min_samples_split, min_samples_leaf, max_features. Na początek jednak skupimy się tylko na max_depth - być może z jego powodu drzewo jest tak mocno przeuczone. Zastosujemy również najlepsze parametry ColumnRemovera znalezione w poprzednim podpunkcie


In [76]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9998, 0.99, 2)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9998, 0.96, 15))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

# od tej pory dane testowe i treningowe będą już wstępnie przeprocesowane
X_train = col_transformer.fit_transform(train_df.drop('target', axis=1), train_df.target)
y_train = train_df.target
X_test = col_transformer.transform(test_df.drop('target', axis=1))
y_test = test_df.target

In [77]:
clf = DecisionTreeClassifier(random_state=42)

parameters = dict(max_depth=np.arange(1, 100))
depth_search = GridSearchCV(clf, cv=3, scoring='roc_auc', return_train_score=True, param_grid=parameters, n_jobs=-1).fit(X_train, y_train)

In [78]:
print(round(depth_search.best_score_, 4), depth_search.best_params_)

0.8008 {'max_depth': 4}


In [79]:
cvres = depth_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(round(mean_score, 4), "   ", params)

0.6825     {'max_depth': 1}
0.7438     {'max_depth': 2}
0.7854     {'max_depth': 3}
0.8008     {'max_depth': 4}
0.8004     {'max_depth': 5}
0.7919     {'max_depth': 6}
0.784     {'max_depth': 7}
0.7719     {'max_depth': 8}
0.7647     {'max_depth': 9}
0.7471     {'max_depth': 10}
0.7353     {'max_depth': 11}
0.7092     {'max_depth': 12}
0.7082     {'max_depth': 13}
0.6954     {'max_depth': 14}
0.669     {'max_depth': 15}
0.6699     {'max_depth': 16}
0.6698     {'max_depth': 17}
0.6551     {'max_depth': 18}
0.6485     {'max_depth': 19}
0.6425     {'max_depth': 20}
0.6273     {'max_depth': 21}
0.6207     {'max_depth': 22}
0.6081     {'max_depth': 23}
0.5975     {'max_depth': 24}
0.5985     {'max_depth': 25}
0.5914     {'max_depth': 26}
0.581     {'max_depth': 27}
0.5752     {'max_depth': 28}
0.5757     {'max_depth': 29}
0.5654     {'max_depth': 30}
0.5616     {'max_depth': 31}
0.5602     {'max_depth': 32}
0.5574     {'max_depth': 33}
0.5581     {'max_depth': 34}
0.5567     {'max_depth': 3

In [80]:
show_scores(depth_search.best_estimator_, X_train, y_train)
show_scores(depth_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23557 |             3 |
|           925 |            15 |

accuracy:              0.9621
precision:             0.8333
recall:                0.016
f1:                    0.0313
roc_auc_discrete:      0.5079
roc_auc_continuous:    0.8135
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         10040 |            10 |
|           450 |             0 |

accuracy:              0.9562
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.4995
roc_auc_continuous:    0.7994


Dość podejrzane te wyniki. Spróbujmy znaleźć parametry z metryką f1

In [81]:
clf = DecisionTreeClassifier(random_state=42)

parameters = dict(max_depth=np.arange(1, 100))
depth_search = GridSearchCV(clf, cv=3, scoring='f1', return_train_score=True, param_grid=parameters, n_jobs=-1).fit(X_train, y_train)

In [82]:
print(round(depth_search.best_score_, 4), depth_search.best_params_)

0.1421 {'max_depth': 29}


In [83]:
cvres = depth_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(round(mean_score, 4), "   ", params)

0.0     {'max_depth': 1}
0.0     {'max_depth': 2}
0.0042     {'max_depth': 3}
0.0082     {'max_depth': 4}
0.0162     {'max_depth': 5}
0.0219     {'max_depth': 6}
0.0288     {'max_depth': 7}
0.0409     {'max_depth': 8}
0.047     {'max_depth': 9}
0.049     {'max_depth': 10}
0.0675     {'max_depth': 11}
0.076     {'max_depth': 12}
0.0827     {'max_depth': 13}
0.0854     {'max_depth': 14}
0.089     {'max_depth': 15}
0.0801     {'max_depth': 16}
0.086     {'max_depth': 17}
0.0932     {'max_depth': 18}
0.106     {'max_depth': 19}
0.1114     {'max_depth': 20}
0.1125     {'max_depth': 21}
0.122     {'max_depth': 22}
0.1213     {'max_depth': 23}
0.112     {'max_depth': 24}
0.1327     {'max_depth': 25}
0.1356     {'max_depth': 26}
0.129     {'max_depth': 27}
0.1317     {'max_depth': 28}
0.1421     {'max_depth': 29}
0.1343     {'max_depth': 30}
0.1357     {'max_depth': 31}
0.1409     {'max_depth': 32}
0.1354     {'max_depth': 33}
0.1382     {'max_depth': 34}
0.1385     {'max_depth': 35}
0.1383   

In [84]:
show_scores(depth_search.best_estimator_, X_train, y_train)
show_scores(depth_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23546 |            14 |
|            29 |           911 |

accuracy:              0.9982
precision:             0.9849
recall:                0.9691
f1:                    0.9769
roc_auc_discrete:      0.9843
roc_auc_continuous:    0.9999
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9641 |           409 |
|           378 |            72 |

accuracy:              0.925
precision:             0.1497
recall:                0.16
f1:                    0.1547
roc_auc_discrete:      0.5597
roc_auc_continuous:    0.5665


Wyniki wyglądają sensowniej niż dla roc_auc, co nie zmienia faktu, że drzewo nadal jest przeuczone i nie dostaliśmy lepszych wyników od braku regulacji. Spróbujemy określić pewną podprzestrzeń parametrów max_depth, min_samples_split, min_samples_leaf, max_features i wykorzystamy klasę RandomizedSearchCV do znalezienia optymalnej kombinacji. Wybierzemy metrykę f1, bo zauważyliśmy już, że dzięki roc_auc algorytm nie wyłapuje żadnych 1.

In [85]:
clf = DecisionTreeClassifier(random_state=42)

parameters = dict(max_depth=np.arange(25, 65, 3), min_samples_split=np.arange(2,8),
                  min_samples_leaf=np.arange(1, 20, 2), max_features=np.arange(20, 150, 5))

# siatka parametrów ma liczność 21840, przeszukujemy więc ok. 45,7% wszystkich możliwości
rand_search = RandomizedSearchCV(clf, scoring='f1', cv=3, return_train_score=True, param_distributions=parameters, n_iter=10000, n_jobs=-1, random_state=42).fit(X_train, y_train)

Sprawdzamy najlepszą kombinację

In [86]:
print(round(rand_search.best_score_, 4), rand_search.best_params_)

0.1182 {'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 125, 'max_depth': 25}


In [87]:
show_scores(rand_search.best_estimator_, X_train, y_train)
show_scores(rand_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23394 |           166 |
|           439 |           501 |

accuracy:              0.9753
precision:             0.7511
recall:                0.533
f1:                    0.6235
roc_auc_discrete:      0.763
roc_auc_continuous:    0.9878
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9778 |           272 |
|           400 |            50 |

accuracy:              0.936
precision:             0.1553
recall:                0.1111
f1:                    0.1295
roc_auc_discrete:      0.542
roc_auc_continuous:    0.5879


Jak na kilka godzin szukania wyniki nie są powalające. Ciężko powiedzieć, czy regularyzacja coś daje w tym przypadku. Ale może chociaż inżynieria cech nam wyjdzie.

### 4.3.3 Dobór zmiennych nie wymagajacy modelu

Korelację uwzględniliśmy w transformatorze ColumnRemover. Technika którą wykorzystamy poniżej to Univariate feature selection - SelectKBest.

### SelectKBest

In [88]:
# szukamy najlepszego parametru k w SelectKBest
clf = Pipeline([
    ('select', SelectKBest()),
    ('model', DecisionTreeClassifier(random_state=42))])

parameters = dict(select__k=np.arange(1, 200, 1))
k_best_search = GridSearchCV(clf, scoring='roc_auc', cv=3, return_train_score=True, param_grid=parameters, n_jobs=-1).fit(X_train, y_train)

In [89]:
# wyniki dla najlepszego znalezionego parametru
print(round(k_best_search.best_score_, 4), k_best_search.best_params_)

0.715 {'select__k': 10}


In [90]:
show_scores(k_best_search.best_estimator_, X_train, y_train)
show_scores(k_best_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|           940 |             0 |

accuracy:              0.9616
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.5
roc_auc_continuous:    0.7189
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         10050 |             0 |
|           450 |             0 |

accuracy:              0.9571
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.5
roc_auc_continuous:    0.7151


Wyniki dla metryki roc_auc ponownie nic nie dają, sprawdzimy f1

In [91]:
clf = Pipeline([
    ('select', SelectKBest()),
    ('model', DecisionTreeClassifier(random_state=42))])

parameters = dict(select__k=np.arange(1, 200, 1))
k_best_search = GridSearchCV(clf, scoring='f1', cv=5, return_train_score=True, param_grid=parameters, n_jobs=-1).fit(X_train, y_train)

print(round(k_best_search.best_score_, 4), k_best_search.best_params_)

0.1531 {'select__k': 106}


In [92]:
show_scores(k_best_search.best_estimator_, X_train, y_train)
show_scores(k_best_search.best_estimator_, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|            15 |           925 |

accuracy:              0.9994
precision:             1.0
recall:                0.984
f1:                    0.992
roc_auc_discrete:      0.992
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9624 |           426 |
|           373 |            77 |

accuracy:              0.9239
precision:             0.1531
recall:                0.1711
f1:                    0.1616
roc_auc_discrete:      0.5644
roc_auc_continuous:    0.5651


Zobaczmy, jak prezentują się wszystkie wyniki

In [93]:
cvres = k_best_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(round(mean_score, 4), "   ", params)

0.0     {'select__k': 1}
0.0     {'select__k': 2}
0.0     {'select__k': 3}
0.0     {'select__k': 4}
0.0     {'select__k': 5}
0.0     {'select__k': 6}
0.0     {'select__k': 7}
0.0     {'select__k': 8}
0.0     {'select__k': 9}
0.0     {'select__k': 10}
0.0     {'select__k': 11}
0.0343     {'select__k': 12}
0.0472     {'select__k': 13}
0.0419     {'select__k': 14}
0.0346     {'select__k': 15}
0.0388     {'select__k': 16}
0.0393     {'select__k': 17}
0.0421     {'select__k': 18}
0.04     {'select__k': 19}
0.0428     {'select__k': 20}
0.0347     {'select__k': 21}
0.0361     {'select__k': 22}
0.0417     {'select__k': 23}
0.0403     {'select__k': 24}
0.0399     {'select__k': 25}
0.0424     {'select__k': 26}
0.0336     {'select__k': 27}
0.0451     {'select__k': 28}
0.046     {'select__k': 29}
0.042     {'select__k': 30}
0.0487     {'select__k': 31}
0.0451     {'select__k': 32}
0.0465     {'select__k': 33}
0.0503     {'select__k': 34}
0.0463     {'select__k': 35}
0.0528     {'select__k': 36}
0.

In [94]:
clf = Pipeline([
    ('select', SelectKBest(k=106)),
    ('model', DecisionTreeClassifier(random_state=42))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|            15 |           925 |

accuracy:              0.9994
precision:             1.0
recall:                0.984
f1:                    0.992
roc_auc_discrete:      0.992
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9624 |           426 |
|           373 |            77 |

accuracy:              0.9239
precision:             0.1531
recall:                0.1711
f1:                    0.1616
roc_auc_discrete:      0.5644
roc_auc_continuous:    0.5651


### 4.3.4 Dobór zmiennych na podstawie modelu

Przeprowadzimy dobór cech na podstawie modelu ze znalezionymi najlepszymi parametrami. Zastosujemy metody SelectFromModel - feature importance, Sequential Feature Selection i RFE

### SelectFromModel - tree-based feature selection

In [95]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

show_scores(clf, X_train, y_train)
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9653 |           397 |
|           379 |            71 |

accuracy:              0.9261
precision:             0.1517
recall:                0.1578
f1:                    0.1547
roc_auc_discrete:      0.5591
roc_auc_continuous:    0.5591


In [96]:
X_train.shape[1]

197

In [97]:
sfl = SelectFromModel(clf, prefit=True)
X_train_t = sfl.transform(X_train)
X_test_t = sfl.transform(X_test)
X_train_t.shape[1]

22

In [98]:
clf.fit(X_train_t, y_train)

show_scores(clf, X_train_t, y_train)
show_scores(clf, X_test_t, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9603 |           447 |
|           381 |            69 |

accuracy:              0.9211
precision:             0.1337
recall:                0.1533
f1:                    0.1429
roc_auc_discrete:      0.5544
roc_auc_continuous:    0.5544


Obserwujemy niewielkie pogorszenie wyników modelu, ale zredukowaliśmy liczbę cech do zaledwie 22!

### Sequential Feature Selection

In [99]:
sfs = SequentialFeatureSelector(
    DecisionTreeClassifier(random_state=42),
    direction='forward',
    scoring='f1',
    n_features_to_select=50,
    cv=3,
    n_jobs=-1)

pipe = Pipeline([
    ('selector', sfs),
    ('model', DecisionTreeClassifier(random_state=42))]).fit(X_train, y_train)

show_scores(pipe, X_train, y_train)

In [100]:
show_scores(pipe, X_test_t, y_test)

Zredukowaliśmy liczbę cech do 50 nie pogarszając wyniku.

### Recursive Feature Elimination

Dla najlepszego modelu drzewa decyzyjnego sprawdźmy które cechy niosą za sobą najwięcej informacji

In [101]:
k_val = [k for k in range (2,100,2)]
score = []
for k in k_val:

    clf = DecisionTreeClassifier(**rand_search.best_params_)
    rfe = RFE(clf,n_features_to_select = k)
    rfe.fit(X_train, y_train)
    X_train_rfe = rfe.transform(X_train)
    X_test_rfe = rfe.transform(X_test)
    clf.fit(X_train_rfe, y_train)

    y_pred_prob = clf.predict_proba(pd.DataFrame(X_test_rfe))

    score_roc = roc_auc_score(y_test, y_pred_prob[:, 1])
    score.append(score_roc)
    print(score_roc, k)

In [102]:
plt.stem(score)
plt.xticks(k_val, score)
plt.xlim([-1, 50])
plt.xlabel("K Values")
plt.ylabel("f1 Score")

![DCT.png](./Img/dct_rfe.png)

Z powyższego wykresu możemy wyczytać, że najlepiej będzie użyć rfe dla 46 cech.
W związku z tym spróbujmy użyć PCA na wybranych kolumnach.

### Principal component analysis

In [103]:
rfe = RFE(DecisionTreeClassifier(**rand_search.best_params_, random_state=42),n_features_to_select = 64).fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

k_values = [i for i in range (2,72)]
scores = []

for k in k_values:

    clf = DecisionTreeClassifier(**rand_search.best_params_)
    pca = PCA(n_components=k)

    X_red_train = pca.fit_transform(X_train_rfe)
    X_red_test = pca.fit_transform(X_test_rfe)

    clf.fit(X_red_train, y_train)
    y_pred_prob = clf.predict_proba(pd.DataFrame(X_red_test))

    score_roc = roc_auc_score(y_test, y_pred_prob[:, 1])
    scores.append(score_roc)


In [104]:
plt.stem(scores)
plt.xticks(k_values, scores)
plt.xlim([-1, 72])
plt.xlabel("K Values")
plt.ylabel("roc_auc Score")
plt.title("PCA for decision Tree")

![DCT.png](./Img/DCT.png)

Widzimy więc, że dla tego modelu najlepiej będzie wybrać wartość n_components = 64

### 4.4 Podsumowanie

Model drzewa decyzyjnego działa bardzo słabo w przypadku naszego zbioru danych i wykazujemy pewną bezradność wobec tego algorytmu nawet po dość szczegółowym i wielokrotnym przeszukiwaniu siatki parametrów. Niemniej jednak udało nam się przeprowadzić selekcję cech niepogarszającą rezultatów modelu. Metody jakie wybraliśmy i ostateczna liczba zmiennych prezentują się następujaco:

- SelectKBest - 106
- Tree-based feature selection (feature_importance) - 22
- Sequential feature selection - 70
- Recursive feature elimination - 46
- RFE + PCA - 64

Zwycięzcą w tym modelu okazało się podejście oparte na modelu i parametrze feature_importance. Trzeba jednak pamiętać, że jest ono skuteczne tylko w przypadku nieprzetrenowanego modelu, czego w tym przypadku nie uzyskaliśmy.


# 5. SVM

## 5.1 Preprocessing

In [105]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10)),
    ('standard_scaler', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

In [ ]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
X_test = test_df.drop('target', axis=1)
y_test = test_df.target

## 5.2 Trening pierwszego modelu

In [108]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', SVC(random_state=42, probability=True))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|           939 |             1 |

accuracy:              0.9617
precision:             1.0
recall:                0.0011
f1:                    0.0021
roc_auc_discrete:      0.5005
roc_auc_continuous:    0.5118


In [109]:
show_scores(clf, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         10050 |             0 |
|           450 |             0 |

accuracy:              0.9571
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.5
roc_auc_continuous:    0.494


Spróbujmy wyciągnąć lepsze wyniki

### 5.2.1 Trening z wybranymi hiperparametrami

In [ ]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

X_train = col_transformer.fit_transform(train_df.drop('target', axis=1), train_df.target)
y_train = train_df.target
X_test = col_transformer.transform(test_df.drop('target', axis=1))
y_test = test_df.target

In [ ]:
svm2 = SVC(kernel='linear', class_weight='balanced', gamma='auto',probability=True, random_state=42).fit(X_train,y_train)

In [ ]:
show_scores(svm2, X_train, y_train)
show_scores(svm2, X_test, y_test)

![svm.png](./img/svm.png)

Pozostaje kwestia strojenia hiperparametrów i wyboru cech dla tego modelu. Niestety, z naszego doświadczenia wiemy, że czas trwania przeliczenia tego dla SVM jest liczony w dniach (co najmniej kilku). Z tego powodu ograniczyliśmy się jedynie do empirycznego wyboru powyższych hiperparametrów.

# 6. Random forest

## 6.1 Preprocessing

In [ ]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

## 6.2 Trening modelu

In [ ]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
X_test = test_df.drop('target', axis=1)
y_test = test_df.target

In [ ]:
rf = RandomForestClassifier(random_state=42, n_estimators=1000)

clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', rf)])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

In [ ]:
show_scores(clf, X_test, y_test)

Model jest bardzo przeuczony, spróbujmy ustawić parametry dla znalezione dla najlepszego pojedynczego drzewa decyzyjnego.

In [ ]:
rf = RandomForestClassifier(random_state=42, n_estimators=1000, **rand_search.best_params_)

clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', rf)])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

In [ ]:
show_scores(clf, X_test, y_test)

# 7. Voting

Zbierzemy teraz najlepsze otrzymane algorytmy regresji logistycznej, drzewa decyzyjnego i svm łącząc je w klasyfikatorze głosującym. Zastosujemy głosowanie miękkie, które lepiej się sprawdzi w naszym przypadku.

In [ ]:
log_reg = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 1, 0)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9996, 0.97, 0)),
            ('min_max', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector_1', SelectKBest(k=33)),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

dct = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 0.99, 2)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9998, 0.96, 15)),
            ('min_max', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector_2', SelectFromModel(DecisionTreeClassifier(random_state=42))),
    ('model', DecisionTreeClassifier(random_state=42))])

svc = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9995, 0.99, 1)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9999, 0.99, 10))]), make_column_selector(dtype_include=np.float64))
    ])),
    ('model', SVC(random_state=42, probability=True))])

In [ ]:
estimators=[('DecisionTree', dct), ('SVM', svc), ('LR', log_reg)]
vc = VotingClassifier(estimators=estimators, voting='soft', weights=[0.2, 0.1, 0.7]).fit(X_train, y_train)

In [39]:
show_scores(vc, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         21333 |          2227 |
|           180 |           760 |

accuracy:              0.9018
precision:             0.2544
recall:                0.8085
f1:                    0.3871
roc_auc_discrete:      0.857
roc_auc_continuous:    0.9434


In [40]:
show_scores(vc, X_test, y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          8944 |          1106 |
|           236 |           214 |

accuracy:              0.8722
precision:             0.1621
recall:                0.4756
f1:                    0.2418
roc_auc_discrete:      0.6828
roc_auc_continuous:    0.7792


Otrzymaliśmy niespotykanie wysoki dotąd rezultat f1 oraz całkiem przyzwoite inne metryki.

# 8. Adaboost

Spróbujmy zastosować Adaboost na pewnych modelach.

In [5]:
from sklearn.ensemble import AdaBoostClassifier

### Regresja Logistyczna

In [ ]:
log_reg = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 1, 0)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9996, 0.97, 0)),
            ('min_max', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector_1', SelectKBest(k=33)),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

In [11]:
scaler = MinMaxScaler()
X_train_logreg = scaler.fit_transform(X_train)
X_test_logreg = scaler.fit_transform(X_test)

In [48]:
ADB_logreg.fit(X_train_logreg, y_train)
show_scores(ADB_logreg, X_train_logreg,y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16137 |          7423 |
|           222 |           718 |

accuracy:              0.688
precision:             0.0882
recall:                0.7638
f1:                    0.1581
roc_auc_discrete:      0.7244
roc_auc_continuous:    0.7998


In [49]:
show_scores(ADB_logreg, X_test_logreg,y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          7167 |          2883 |
|           142 |           308 |

accuracy:              0.7119
precision:             0.0965
recall:                0.6844
f1:                    0.1692
roc_auc_discrete:      0.6988
roc_auc_continuous:    0.7702


### Drzewa decyzyjne

In [ ]:
dct = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 0.99, 2)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9998, 0.96, 15)),
            ('min_max', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector_2', SelectFromModel(DecisionTreeClassifier(random_state=42))),
    ('model', DecisionTreeClassifier(random_state=42))])

In [24]:
ADB_dct = AdaBoostClassifier(base_estimator=clf, n_estimators=1000,learning_rate=0.5,
                         algorithm='SAMME.R', random_state=42)

In [25]:
ADB_dct.fit(X_train, y_train)
show_scores(ADB_dct, X_train,y_train)
show_scores(ADB_dct, X_test,y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         10032 |            18 |
|           443 |             7 |

accuracy:              0.9561
precision:             0.28
recall:                0.0156
f1:                    0.0295
roc_auc_discrete:      0.5069
roc_auc_continuous:    0.6767


In [26]:
ADB_dct = AdaBoostClassifier(base_estimator=clf, n_estimators=2000,learning_rate=0.5,
                         algorithm='SAMME.R', random_state=42)

In [27]:
ADB_dct.fit(X_train, y_train)
show_scores(ADB_dct, X_train,y_train)
show_scores(ADB_dct, X_test,y_test)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23559 |             1 |
|             2 |           938 |

accuracy:              0.9999
precision:             0.9989
recall:                0.9979
f1:                    0.9984
roc_auc_discrete:      0.9989
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         10031 |            19 |
|           443 |             7 |

accuracy:              0.956
precision:             0.2692
recall:                0.0156
f1:                    0.0294
roc_auc_discrete:      0.5068
roc_auc_continuous:    0.6844


Jak widać, adaboost może pomóc w drzewach decyzyjnych, ponieważ znacznie poprawia roc_auc_continuous. Jednakże dla regresji logistycznej jest on niepotrzebny, wręcz niewskazany

# 9. Podsumowanie projektu

W powyższym projekcie rozpoczęliśmy z 300 zmiennymi i jedną zmienną przewidywaną. 

Na początku zajeliśmy się preprocessingiem. Postanowiliśmy podzielić go na zmienne całkowitoliczbowe i zmiennoprzecinkowe. Wyrzuciliśmy zmienne noszące takie same dane, zmienne skorelowane, stałe i nie noszące informacji. 


Następnie zajęliśmy się modelami. Postanowiliśmy oceniać wyniki na podstawie polem pod krzywą roc_auc. Wyniki wyglądają następująco:

|Model   |No_hyper   |Col_rem   |with_hyper   |PCA   |K_best   |SFS   |num_rfe   |num_PCA   |
|---|---|---|---|---|---|---|---|---|
|Lin_regrr   |0.806   |0.777   |0.815   |0.738   |   39 / 114 | 50   |72   |3|
|dec_tree   |0.555   |0.780   |0.799   |0.682   |117 |40 |22 | 64|

|Model   |No_hyper   |with_hyper   |
|---|---|---|
|SVC   |0.670   |0.768   |
|KNN   |0.542 |0.570   |
|rand_forest   |0.796   | 0.800   |
|AdaBoost logreg  |0.774   |0.799   |
|AdaBoost dct  |0.676   |0.684   |
|Voting   |   |   | 


Jak możemy zauważyć, funkcje które zmniejszały liczbę cech pogarszały wyniki modeli. Postanowiliśmy je jednak pokazać, ponieważ poprawa wydajności którą niesie ze sobą stosowanie tych funkcji znacznie pomaga w obliczeniach.